## Data preparation
*Tim Braams (8460701), Vinh Phan (8462380), Maximilian Pintilie (8462780), Rahul Singh (8464147), Kartik Vijay (8463465), Diego Zucchino (8345420)*   
  
Currently we have minute data for 10 cryptos. To reduce the data size we convert the data to hourly by taking the volume weighted average price of that hour.

In [ ]:
import pandas as pd
import os
import tqdm
import scripts.utils as ut

In [ ]:
# Data not provided with this submission, please put it into Crypto_data_minute folder in this directory
files =  [file for file in os.listdir("data/Crypto_data_minute") if file.split(".")[1] == "txt"]

#### Data resampling

##### Equidistant data
First we create frames for each crypto that have a continous timeseries (equidistant timestamps). This data will be used for the Darts libary. If a hour had no trades, we forward fill the last observed price for the hour without prices.

In [ ]:
for file in files: 
    name = file.split(".")[0].split("_")[0]
    df_full = pd.read_csv(f"data/Crypto_data_minute/{file}", header=None)
    df_full.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume"]
    df_full = df_full.sort_values("Open Time")
    df_full["Open Time"] = pd.to_datetime(df_full["Open Time"])
    df_full["volume_weighted"] = df_full["Close"] * df_full["Volume"]
    df_resample = df_full.resample("1h", on="Open Time").sum()
    df_resample["wClose"] = df_resample["volume_weighted"] / df_resample["Volume"]
    df_resample = df_resample.fillna(method="ffill")
    df_resample.to_json(f"data/Resample/{name}.json")

In [ ]:
master_table = ut.read_and_concat("data/Resample")
master_table.to_csv("data/master_returns_df.csv", index=False)

##### Raw data (non-equidistant)
Next we do the same but without forward filling. Therefore, if a hour had no trades, it will not show up in the dataframe.

In [ ]:
for file in files: 
    name = file.split(".")[0].split("_")[0]
    df_full = pd.read_csv(f"data/Crypto_data_minute/{file}", header=None)
    df_full.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume"]
    df_full = df_full.sort_values("Open Time")
    df_full["Open Time"] = pd.to_datetime(df_full["Open Time"])
    df_full["volume_weighted"] = df_full["Close"] * df_full["Volume"]
    df_resample = df_full.resample("1h", on="Open Time").sum()
    df_resample["wClose"] = df_resample["volume_weighted"] / df_resample["Volume"]
    df_resample = df_resample.dropna(subset=["wClose"])
    df_resample.to_json(f"data/Resample_non_equidistant/{name}.json")

#### Data pooling

In [1]:
import tqdm
import datetime as dt

In [ ]:
files =  [file for file in os.listdir("data/Resample_non_equidistant") if file.split(".")[1] == "json"]

The pooled data will have a column for returns and a window of returns preceding that timestamp (each timestamp in the window has one column t1, ..., tn). One final column marks the coin the table is generated for. In the end all cryptos are concatiated into one big frame. This looks like to following:

|            | returns | t1   | t2   | t3   | asset |
|------------|---------|------|------|------|-------|
| 2022-01-01 | 0.2     | 0.1  | 0.05 | 0.03 | BTC   |
| 2022-01-02 | 0.1     | 0.2  | 0.1  | 0.05 | BTC   |
| 2022-01-03 | 0.15    | 0.1  | 0.2  | 0.1  | BTC   |
| 2022-01-01 | 0.05    | 0.01 | 0.5  | 0.33 | ETH   |
| 2022-01-02 | 0.3     | 0.05 | 0.01 | 0.5  | ETH   |
| 2022-01-03 | 0.1     | 0.3  | 0.05 | 0.01 | ETH   |

All returns are calculted in relation to the next timestamp (i.e. about hourly returns - if all timestamps are available). The window was choosen based on EDA (next notebook).

In [ ]:
def get_returns(df, timedelta: int = 1):
    df["returns"] = ((df["wClose"] / df["wClose"].shift(timedelta)) - 1)
    return df.dropna(subset=["returns"])

In [ ]:
def get_shifted_frame(file, window=100, norm=True):
    name = file.split(".")[0]
    df_full = pd.read_json(f"data/Resample_non_equidistant/{name}.json")
    df_temp = get_returns(df_full, timedelta=1).copy()
    train, test = get_train_test(df_temp)
    if norm:
        train_norm = pd.DataFrame((train["returns"]-train["returns"].mean())/train["returns"].std())
        test_norm = pd.DataFrame((test["returns"]-test["returns"].mean())/test["returns"].std())
        
        train_norm["mean"] = train["returns"].mean()
        train_norm["std"] = train["returns"].std()
        test_norm["mean"] = test["returns"].mean()
        test_norm["std"] = test["returns"].std()
        
        returns = pd.concat([train_norm, test_norm])
        returns.index = df_temp.index
        
    else:
        returns = df_temp["returns"]
    
    if norm:
        columns = [returns["returns"], returns["mean"], returns["std"]]
    else: 
        columns = [returns]
    for n in range(window):
        if norm:
            series_new = returns["returns"].shift(n + 1)
        else:
            series_new = returns.shift(n + 1)
        series_new.name = f"t{n+1}"
        columns.append(series_new)
    concat_frame = pd.concat(columns, axis=1)
    concat_frame["asset"] = name
    return concat_frame

def get_train_test(frame):
    cutoff_ts = frame.index[-1] - pd.Timedelta(365, "days")
    train = frame[frame.index < cutoff_ts].copy()
    test = frame[frame.index >= cutoff_ts].copy()
    return train, test

We choose to not normalize the data, as we observed in the EDA that the returns are mean centred and therefore normilazation would not greatly benefit the models. Individual models might have normilazation implemented (see simple LSTM), for example as a layer, if the model benefitted from it.

In [ ]:
frames_train = []
frames_test = []
for file in files:
    print(file)
    frame = get_shifted_frame(file=file, window=300, norm=False)
    train, test = get_train_test(frame)
    frames_train.append(train)
    frames_test.append(test) 

Save the data.

In [ ]:
train_full = pd.concat(frames_train, axis=0).dropna()
test_full = pd.concat(frames_test, axis=0).dropna()

train_full.reset_index().to_csv("data/pooled_train_300.csv")
test_full.reset_index().to_csv("data/pooled_test_300.csv")
train_full.reset_index().to_json("data/pooled_train_300.json", date_unit="ns")
test_full.reset_index().to_json("data/pooled_test_300.json", date_unit="ns")